## Useful functions for DNA Assembly
1. k-mers contruction
2. Overlap (suffix-prefix) 
3. Overlap ALL 

### 1.- *k-mer* is a subsequence of length *k* useful to build index for DNA assembly methods such as de Bruijn graphs 

In [57]:

def kmers(reads,k):
    kmers={}
    j=-1
    for read in reads:
        j+=1
        for i in range(len(read)-k+1):
            kmer=read[i:i+k]
            if not kmer in kmers:
                kmers[kmer]=[j]
            else:
                kmers[kmer].append(j)

    return kmers

#kmers(reads,4)
#overlap(kmers,reads,min_length=3)

*k-mer* example

In [28]:
reads = ['CGTACG', 'TACGTA','TTTAAA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
kmers4 = kmers(reads,5)
kmers4
#reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
#naive_overlap_map(reads,5)

{'CGTAC': [0, 4],
 'GTACG': [0, 3, 5],
 'TACGT': [1, 3],
 'ACGTA': [1, 4],
 'TTTAA': [2],
 'TTAAA': [2],
 'TACGA': [5, 6],
 'ACGAT': [6]}

### 2. **Overlap** function return the length of the longest suffix of *a* matching a prefix of *b*

In [42]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match
        

In [52]:
read ='GTTACGT'
read1 ='ACGTAC'
read2 ='CGTACGA'
read3 ='TACGAT'
overlap(read,read2)
#Only matches if prefix of b overlaps sufix of a

3

In [81]:
def naive_overlap_map(reads, k):
    from itertools import permutations
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            olaps[(a, b)] = olen
    return olaps

In [83]:
reads = ['CGTACG', 'TACGTA','TTTAAA', 'GTTCGT', 'ACGTAC', 'GTACGA', 'TACGAT']
print(naive_overlap_map(reads, 3))

{('CGTACG', 'TACGTA'): 4, ('CGTACG', 'ACGTAC'): 3, ('CGTACG', 'GTACGA'): 5, ('CGTACG', 'TACGAT'): 4, ('TACGTA', 'CGTACG'): 4, ('TACGTA', 'ACGTAC'): 5, ('TACGTA', 'GTACGA'): 3, ('GTTCGT', 'CGTACG'): 3, ('ACGTAC', 'CGTACG'): 5, ('ACGTAC', 'TACGTA'): 3, ('ACGTAC', 'GTACGA'): 4, ('ACGTAC', 'TACGAT'): 3, ('GTACGA', 'TACGAT'): 5}


### Overlap_ALL will find the overlaps only in those reads that share *kmer* of length *k*
The advantage of this implementation is that the search for overlaps will be faster

In [84]:
def overlapAll(reads, k):
    from itertools import permutations
    olaps = {}
    kmerS = kmers(reads,k)
    vals = list(kmerS.values())
    lenss=[len(vals[val]) for val in range(len(vals))]
    bla = list(enumerate(lenss))
    newList = set()
    for bl in range(len(bla)):
        if bla[bl][1] > 1:
            n = newList.update(vals[bla[bl][0]])

    for (idx_a,a), (idx_b,b) in permutations(enumerate(newList),2):
        olen = overlap(reads[a], reads[b], min_length=k)
        if olen > 0:
            olaps[(reads[a], reads[b])] = olen
    
    nodes = len(newList)
    return olaps,nodes        

In [88]:
reads = ['CGTACG', 'TACGTA','TTTAAA', 'GTTCGT', 'ACGTAC', 'GTACGA', 'TACGAT']
overlapAll(reads,4)


({('CGTACG', 'TACGTA'): 4,
  ('CGTACG', 'GTACGA'): 5,
  ('CGTACG', 'TACGAT'): 4,
  ('TACGTA', 'CGTACG'): 4,
  ('TACGTA', 'ACGTAC'): 5,
  ('ACGTAC', 'CGTACG'): 5,
  ('ACGTAC', 'GTACGA'): 4,
  ('GTACGA', 'TACGAT'): 5},
 5)

In [74]:
reads = ['CGTACG', 'TACGTA','TTTAAA', 'GTTCGT', 'ACGTAC', 'GTACGA', 'TACGAT']
kmers4 = kmers(reads,4)
vals = list(kmers4.values())
lenss=[len(vals[val]) for val in range(len(vals))]
bla = list(enumerate(lenss))

newList = set()
newReads = []
for bl in range(len(bla)):
    if bla[bl][1] > 1:
        n = newList.update(vals[bla[bl][0]])
        
newList
from itertools import permutations
list(permutations(newList,2))
#(idx_a, a), (idex_b,b) in permutations(enumerate(newList),2)

[(0, 1),
 (0, 4),
 (0, 5),
 (0, 6),
 (1, 0),
 (1, 4),
 (1, 5),
 (1, 6),
 (4, 0),
 (4, 1),
 (4, 5),
 (4, 6),
 (5, 0),
 (5, 1),
 (5, 4),
 (5, 6),
 (6, 0),
 (6, 1),
 (6, 4),
 (6, 5)]

In [77]:
from itertools import permutations

def naive_overlap_map(reads, k):
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            olaps[(a, b)] = olen
    return olaps

In [89]:
reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
qq = overlapAll(reads, 4)
pp = list(qq.keys())
nodes = set()
[nodes.update(pp[p]) for p in range(len(pp))]
nodes

AttributeError: 'tuple' object has no attribute 'keys'

In [64]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            qual = fh.readline().rstrip() #base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences

In [66]:
seqs=readFastq('ERR266411_1.for_asm.fastq')

In [67]:
q3, nodes = overlapAll(seqs,30)
#print(nodes, len(q3))
#edges= len(q3)= 904746
nodes


9960

In [90]:
pp = list(q3.keys())
nodes = set()
[nodes.update(pp[p]) for p in range(len(pp))]
len(nodes)

9750

In [91]:

def overlapAll_nodes(reads, k):
    from itertools import permutations
    olaps = {}
    kmerS = kmers(reads,k)
    vals = list(kmerS.values())
    lenss=[len(vals[val]) for val in range(len(vals))]
    bla = list(enumerate(lenss))
    newList = set()
    for bl in range(len(bla)):
        if bla[bl][1] > 1:
            n = newList.update(vals[bla[bl][0]])
           
    nodes = len(newList)    
    return nodes    

overlapAll_nodes(seqs,30)

9960